In [1]:
import os
import cv2
import numpy as np

In [2]:
def splitImg(img, p, q):
    blocks = []
    height , width = img.shape[:2]
   
    rows = width // q
    columns = height // p
    for column in range(columns):
        for row in range(rows):
                left = row * q
                upper = column * p
                right = left + q
                lower = upper + p
                block = img[upper:lower, left:right]
                blocks.append(block)
    return blocks

In [3]:
def constantAreaCoding(img, p, q):    
    blocks = splitImg(img, p, q)
    codes = ['0','11','10']
    code = ""
    for block in blocks:
        if np.all(block == 0):
            code += codes[0]
        elif np.all(block == 255):
            code += codes[1]
        else:
            code += codes[2] 
            for row in block:
                for pixel in row:
                    if pixel == 255:
                        code += '1' 
                    else:
                        code += pixel.astype(str)
    
    compressionRatio = img.size/len(code)
    dataRedundancy = 1 - 1/compressionRatio
    
    return  p ,q , img.shape[0], img.shape[1],  code , compressionRatio , dataRedundancy

In [4]:
def constantAreaDecoding(code, p, q, height, width):
    codes = ['0','11','10']
    blocks = []
    index = 0
    while index in range(len(code)) :
        if code[index] == codes[0]:
            block = np.zeros((p, q), dtype=np.uint8)
            index=index+1
            
        elif code[index:index+2] == codes[1]:
            block = np.ones((p, q), dtype=np.uint8)
            index = index+2
            
        elif code[index:index+2] == codes[2]: 
            index=index+2
            block = np.zeros((p, q), dtype=np.uint8)
            
            for row in range(0,p):
                for column in range(0,q):
                    block[row][column] = int(code[index])
                    index = index+1
                    
        blocks.append(block * 255)
        
    return blocks

In [5]:
def imageReconstruction(code, p, q, height, width):
    img = np.ones((height, width), dtype=np.uint8) * 255
    blocks = constantAreaDecoding(code, p, q, height, width)    
    x = 0
    y = 0
    index = 0
  
    while y < height:
        while x < width:
            block = blocks[index]
            for row in range(p):
                for column in range(q):
                    img[y + row][x + column] = block[row][column]
            index += 1
            x += q
        y += p
        x = 0
    
    return img

In [6]:
def populate(height, width, generations):
    population = []
    while len(population) < generations:
        
        p = np.random.randint(1,  height)
        q = np.random.randint(1,  width)
        if height % p == 0 and width % q == 0: population.append([p,q])
            
    return population

In [7]:
def fitness(img, population):
    
    fitness = []
    for chromosome in population:
            fitness.append(constantAreaCoding(img, chromosome[0], chromosome[1])[5])
            
    population , fitness = zip(*sorted(zip(population, fitness), key=lambda x: x[1], reverse=True))
    return list(population),list(fitness)


In [8]:
def binarize(pBits, qBits, chromosome):
    p, q = chromosome
    p = format(p, f'0{pBits}b')
    q = format(q, f'0{qBits}b')
    return p + q 

In [9]:
def decimate(pBits, qBits, chromosome):
    p = int(str(chromosome[:pBits]),2)
    q = int(str(chromosome[pBits:]),2)
    return [p,q]    

In [10]:
def crossover(parents):
    crossoverPoint = np.random.randint(0,len(parents[0]))
    children = []
    children.append(parents[0][:crossoverPoint] + parents[1][crossoverPoint:])
    children.append(parents[1][:crossoverPoint] + parents[0][crossoverPoint:])
    return children

In [11]:
def mutate(chromosome,mutationProbability):
    if np.random.random() >= mutationProbability:
        mutationPoint = np.random.randint(0,len(chromosome))
        chromosome = str(chromosome[:mutationPoint]) + str(1-int(chromosome[mutationPoint])) + str(chromosome[mutationPoint+1:])
    return chromosome

In [12]:
def Genetic(img, generations, mutationProbability):
    height, width = img.shape
    pBits = len(bin(height)[2:]) 
    qBits = len(bin(width)[2:])
    population = populate(height, width, generations)
    population, fitnesses = fitness(img, population)
    bestCompressionRatio = 0
    bestBlockSizes = None
    while bestCompressionRatio != fitnesses[0]:
        for generation in range(0, int(generations / 2), 2):
            children = crossover( [ 
                                  binarize(pBits,qBits,population[generation]), 
                                  binarize(pBits,qBits,population[generation + 1]) 
                                  ] )
            population.append(decimate(pBits, qBits, mutate(children[0], mutationProbability)))
            population.append(decimate(pBits, qBits, mutate(children[1], mutationProbability)))
        population, fitnesses = fitness(img, population)
        population = population[:generations]
        fitnesses = fitnesses[:generations]
        if fitnesses[0] > bestCompressionRatio :
            bestCompressionRatio = fitnesses[0]
            bestBlockSizes = population[0]
            
    print(f'Best size : {bestBlockSizes[0]} * {bestBlockSizes[1]}') 
    print(f'Best CR: {bestCompressionRatio}')
    
    return bestBlockSizes, bestCompressionRatio

In [13]:
def saveCompressedImage(compression, index):
    p, q, height, width, code = compression
    with open(f'images/compressed{i}', 'w') as file:
        file.write(f'p x q: {p} x {q}\n')
        file.write(f'height: {height}\n')
        file.write(f'width: {width}\n')
        file.write(f'CAC: {code}\n')

In [14]:
def loadCompressionElements(compressedFile):
    compression = {}
    with open(f'images/{compressedFile}', 'r') as file:
        lines = file.readlines()
        for line in lines:
            key, value = line.strip().split(': ')
            compression[key] = value
    
    p, q = map(int, compression['p x q'].split(' x '))
    height = int(compression['height'])
    width = int(compression['width'])
    code = compression['CAC']
    
    return p, q, height, width, code

In [15]:
images = [file for file in os.listdir('images') if file.endswith(".jpg")]
for image, i in zip(images, range(len(images))):
    img = cv2.imread(f'images/{image}')
    cv2.namedWindow('Original Image', cv2.WINDOW_NORMAL)
    cv2.imshow('Original Image', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    binaryImg = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    (p,q) , cR = Genetic(binaryImg, 5, 0.4)
    compression = constantAreaCoding(binaryImg, p, q)[:5]
    saveCompressedImage(compression, i)

compressed = [file for file in os.listdir('images') if file.startswith("compressed")]
for file in compressed:
    p, q, height, width, code = loadCompressionElements(file)
    img = imageReconstruction(code, p, q, height, width)
    cv2.imshow('DeCompressed Binary img', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

Best size : 1 * 80
Best CR: 4.742386117136659
Best size : 7 * 32
Best CR: 4.470692165491518
